In [ ]:
import pygame
import random
import platform
import asyncio

pygame.init()
pygame.mixer.init(frequency=44100, size=-16, channels=2, buffer=512)

# Screen settings
WIDTH, HEIGHT = 800, 600
win = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Space Invaders")

# FPS
clock = pygame.time.Clock()
FPS = 60

# Colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GRAY = (100, 100, 100)
YELLOW = (255, 255, 0)

# Sizes
SHIP_WIDTH, SHIP_HEIGHT = 64, 64
ENEMY_SIZE = 64
BOSS_SIZE = 128
BULLET_WIDTH, BULLET_HEIGHT = 8, 20
ENEMY_BULLET_WIDTH, ENEMY_BULLET_HEIGHT = 6, 15

# Load images and sounds
try:
    spaceship_img = pygame.transform.scale(pygame.image.load("spaceship.png"), (SHIP_WIDTH, SHIP_HEIGHT))
    enemy_img = pygame.transform.scale(pygame.image.load("enemy.png"), (ENEMY_SIZE, ENEMY_SIZE))
    enemy2_img = pygame.transform.scale(pygame.image.load("enemy2.png"), (ENEMY_SIZE, ENEMY_SIZE))
    boss_img = pygame.transform.scale(pygame.image.load("boss.png"), (BOSS_SIZE, BOSS_SIZE))
    bullet_img = pygame.transform.scale(pygame.image.load("bullet.png"), (BULLET_WIDTH, BULLET_HEIGHT))
    background_img = pygame.transform.scale(pygame.image.load("background.jpg"), (WIDTH, HEIGHT))
    shoot_sound = pygame.mixer.Sound("shoot.wav")
    explosion_sound = pygame.mixer.Sound("explosion.wav")
    boss_explosion_sound = pygame.mixer.Sound("boss_explosion.wav")
    pygame.mixer.music.load("background_music.wav")
except FileNotFoundError as e:
    print(f"Error: File not found - {e}")
    pygame.quit()
    exit()

# Font
font = pygame.font.SysFont("arial", 36)
title_font = pygame.font.SysFont("arial", 64)

# Classes
class Spaceship:
    def __init__(self):
        self.x = WIDTH // 2 - SHIP_WIDTH // 2
        self.y = HEIGHT - SHIP_HEIGHT - 10
        self.speed = 6
        self.health = 3

    def draw(self):
        win.blit(spaceship_img, (self.x, self.y))
        # Draw health bar
        pygame.draw.rect(win, RED, (self.x, self.y - 20, SHIP_WIDTH, 10))
        pygame.draw.rect(win, (0, 255, 0), (self.x, self.y - 20, SHIP_WIDTH * (self.health / 3), 10))

    def move(self, keys):
        if keys[pygame.K_LEFT] and self.x > 0:
            self.x -= self.speed
        if keys[pygame.K_RIGHT] and self.x < WIDTH - SHIP_WIDTH:
            self.x += self.speed

class Enemy:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.speed_x = 1.5

    def draw(self):
        win.blit(enemy_img, (self.x, self.y))

class Enemy2:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.speed_x = 1.5

    def draw(self):
        win.blit(enemy2_img, (self.x, self.y))

class Boss:
    def __init__(self):
        self.x = WIDTH // 2 - BOSS_SIZE // 2
        self.y = 50
        self.health = 5
        self.speed_x = 2

    def draw(self):
        win.blit(boss_img, (self.x, self.y))
        pygame.draw.rect(win, RED, (self.x, self.y - 20, BOSS_SIZE, 10))
        pygame.draw.rect(win, (0, 255, 0), (self.x, self.y - 20, BOSS_SIZE * (self.health / 5), 10))

class Bullet:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.speed = 8

    def draw(self):
        win.blit(bullet_img, (self.x, self.y))

    def move(self):
        self.y -= self.speed

class EnemyBullet:
    def __init__(self, x, y, is_boss=False):
        self.x = x
        self.y = y
        self.speed = 5
        self.is_boss = is_boss

    def draw(self):
        color = (255, 50, 50) if self.is_boss else YELLOW
        pygame.draw.rect(win, color, (self.x, self.y, ENEMY_BULLET_WIDTH, ENEMY_BULLET_HEIGHT))

    def move(self):
        self.y += self.speed

# Function to create enemies
def create_enemies(count):
    enemies = []
    enemy2_count = count // 3  # Roughly 1/3 of enemies are Enemy2
    enemy1_count = count - enemy2_count
    fixed_y = 50  # All enemies in a single horizontal row
    spacing = WIDTH // count  # Even spacing across the screen

    # Place Enemy2 (shooting) on the left
    for i in range(enemy2_count):
        new_x = i * spacing + (spacing // 2 - ENEMY_SIZE // 2)
        enemies.append(Enemy2(new_x, fixed_y))

    # Place Enemy (non-shooting) on the right
    for i in range(enemy1_count):
        new_x = (i + enemy2_count) * spacing + (spacing // 2 - ENEMY_SIZE // 2)
        enemies.append(Enemy(new_x, fixed_y))

    return enemies

# Game over screen
def show_game_over():
    try:
        running = True
        while running:
            win.blit(background_img, (0, 0))
            game_over_text = title_font.render("Game Over", True, WHITE)
            game_over_rect = game_over_text.get_rect(center=(WIDTH // 2, HEIGHT // 3))
            win.blit(game_over_text, game_over_rect)
            return_text = font.render("Press ESC to return to menu", True, WHITE)
            return_rect = return_text.get_rect(center=(WIDTH // 2, HEIGHT // 2))
            win.blit(return_text, return_rect)

            pygame.display.update()

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    return False
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        return True
            clock.tick(FPS)
    except Exception as e:
        print(f"Error in game over screen: {e}")
        return True  # Return to menu to prevent crash
    return True

# Win screen
def show_win():
    try:
        running = True
        while running:
            win.blit(background_img, (0, 0))
            win_text = title_font.render("You Win!", True, WHITE)
            win_rect = win_text.get_rect(center=(WIDTH // 2, HEIGHT // 3))
            win.blit(win_text, win_rect)
            return_text = font.render("Press ESC to return to menu", True, WHITE)
            return_rect = return_text.get_rect(center=(WIDTH // 2, HEIGHT // 2))
            win.blit(return_text, return_rect)

            pygame.display.update()

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    return False
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        return True
            clock.tick(FPS)
    except Exception as e:
        print(f"Error in win screen: {e}")
        return True  # Return to menu to prevent crash
    return True

# Menu class
class Menu:
    def __init__(self):
        self.options = ["Start Game", "Instructions", "Exit"]
        self.selected = 0

    def draw(self):
        win.blit(background_img, (0, 0))
        title_text = title_font.render("Space Invaders", True, WHITE)
        title_rect = title_text.get_rect(center=(WIDTH // 2, HEIGHT // 4))
        win.blit(title_text, title_rect)

        for i, option in enumerate(self.options):
            color = WHITE if i == self.selected else GRAY
            text = font.render(option, True, color)
            text_rect = text.get_rect(center=(WIDTH // 2, HEIGHT // 2 + i * 60))
            win.blit(text, text_rect)

    def update_selection(self, direction):
        self.selected = (self.selected + direction) % len(self.options)

    def select(self):
        return self.options[self.selected]

# Instructions screen
def show_instructions():
    try:
        running = True
        while running:
            win.blit(background_img, (0, 0))
            instructions = [
                "Instructions:",
                "Use LEFT and RIGHT arrows to move",
                "Press SPACE to shoot",
                "Defeat all enemies and the boss to win",
                "Avoid enemy and boss bullets",
                "Lose after 3 hits",
                "Press ESC to return to menu"
            ]
            for i, line in enumerate(instructions):
                text = font.render(line, True, WHITE)
                text_rect = text.get_rect(center=(WIDTH // 2, HEIGHT // 4 + i * 50))
                win.blit(text, text_rect)

            pygame.display.update()

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    return False
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        return True
            clock.tick(FPS)
    except Exception as e:
        print(f"Error in instructions screen: {e}")
        return True  # Return to menu to prevent crash
    return True

# Game loop
async def game_loop():
    try:
        spaceship = Spaceship()
        enemies = create_enemies(10)
        bullets = []
        enemy_bullets = []
        boss = None

        running = True
        while running:
            clock.tick(FPS)
            win.blit(background_img, (0, 0))

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    return False
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        return True

            keys = pygame.key.get_pressed()
            spaceship.move(keys)

            # Shooting
            if keys[pygame.K_SPACE]:
                if len(bullets) < 50:
                    bullets.append(Bullet(spaceship.x + SHIP_WIDTH // 2 - BULLET_WIDTH // 2, spaceship.y))
                    shoot_sound.play()

            spaceship.draw()

            # Enemy logic
            move_down = False
            for enemy in enemies:
                if enemy.x <= 0 or enemy.x >= WIDTH - ENEMY_SIZE:
                    move_down = True
                    break

            for enemy in enemies:
                if move_down:
                    enemy.speed_x *= -1
                    enemy.y += 20
                enemy.x += enemy.speed_x
                enemy.draw()
                # Enemy2 shooting
                if isinstance(enemy, Enemy2) and random.random() < 0.005:  # 0.5% chance per frame
                    enemy_bullets.append(EnemyBullet(enemy.x + ENEMY_SIZE // 2, enemy.y + ENEMY_SIZE))

            # Spawn boss when no enemies
            if len(enemies) == 0 and boss is None:
                boss = Boss()

            # Boss logic
            if boss:
                boss.x += boss.speed_x
                if boss.x <= 0 or boss.x >= WIDTH - BOSS_SIZE:
                    boss.speed_x *= -1
                boss.draw()
                if random.random() < 0.01:
                    enemy_bullets.append(EnemyBullet(boss.x + BOSS_SIZE // 2, boss.y + BOSS_SIZE, is_boss=True))

            # Enemy and boss bullets
            for e_bullet in enemy_bullets[:]:
                e_bullet.move()
                e_bullet.draw()
                if spaceship.x < e_bullet.x < spaceship.x + SHIP_WIDTH and spaceship.y < e_bullet.y < spaceship.y + SHIP_HEIGHT:
                    spaceship.health -= 1
                    enemy_bullets.remove(e_bullet)
                    explosion_sound.play()
                    if spaceship.health <= 0:
                        return show_game_over()  # Return result of game over screen
                elif e_bullet.y > HEIGHT:
                    enemy_bullets.remove(e_bullet)

            # Player bullets
            for bullet in bullets[:]:
                bullet.move()
                bullet.draw()
                for enemy in enemies[:]:
                    if enemy.x < bullet.x < enemy.x + ENEMY_SIZE and enemy.y < bullet.y < enemy.y + ENEMY_SIZE:
                        enemies.remove(enemy)
                        bullets.remove(bullet)
                        explosion_sound.play()
                        break
                if boss:
                    if boss.x < bullet.x < boss.x + BOSS_SIZE and boss.y < bullet.y < boss.y + BOSS_SIZE:
                        boss.health -= 1
                        bullets.remove(bullet)
                        if boss.health <= 0:
                            boss_explosion_sound.play()
                            return show_win()  # Return result of win screen
                        break
                if bullet.y < 0:
                    bullets.remove(bullet)

            pygame.display.update()
            await asyncio.sleep(1.0 / FPS)
    except Exception as e:
        print(f"Error in game loop: {e}")
        return True  # Return to menu to prevent crash
    return True

# Main menu loop
async def main():
    try:
        menu = Menu()
        pygame.mixer.music.play(-1)
        pygame.mixer.music.set_volume(0.5)
        running = True
        while running:
            clock.tick(FPS)
            menu.draw()

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_UP:
                        menu.update_selection(-1)
                    if event.key == pygame.K_DOWN:
                        menu.update_selection(1)
                    if event.key == pygame.K_RETURN:
                        selected_option = menu.select()
                        if selected_option == "Start Game":
                            continue_game = await game_loop()
                            if not continue_game:
                                running = False
                        elif selected_option == "Instructions":
                            continue_menu = show_instructions()
                            if not continue_menu:
                                running = False
                        elif selected_option == "Exit":
                            running = False

            pygame.display.update()
            await asyncio.sleep(1.0 / FPS)

        pygame.mixer.music.stop()
        pygame.quit()
    except Exception as e:
        print(f"Error in main loop: {e}")
        pygame.mixer.music.stop()
        pygame.quit()

# Run the game
if platform.system() == "Emscripten":
    asyncio.ensure_future(main())
else:
    if __name__ == "__main__":
        try:
            loop = asyncio.get_running_loop()
            loop.create_task(main())
        except RuntimeError:
            asyncio.run(main())